# Goals
In this colab you will:

*   Learn how genomes are stored as data.
*   Visualize the genomes using two tools - Multiple Sequence Alignment Viewer and UCSC Genome Browser.
*   Understand how to read and parse a genome in python.
*   Make your own discoveries about the SARS-CoV-2 genome.


<img src="https://www.publicdomainpictures.net/pictures/50000/velka/dna.jpg" alt="drawing" width="500"/>

In [ ]:
#@title #Set up the notebook
!pip install Biopython
from Bio import SeqIO
import numpy as np
import pandas as pd
from collections import Counter
import gdown

# data_path = 'https://drive.google.com/uc?id=1PqupjbA0HYNs1fd1y6TAkkvZeMQ6kmK0'
cov2_vs_rtg13 = './SARS_CoV_2_vs_RTG13.fasta'
# gdown.download(data_path, cov2_vs_rtg13, True)

!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/SARS_CoV_2_vs_RTG13.fasta'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-07-13 17:21:37--  https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/SARS_CoV_2_vs_RTG13.fasta
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.126.128, 74.125.132.128, 173.194.196.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60953 (60K) [application/octet-stream]
Saving to: ‘SARS_CoV_2_vs_RTG13.fasta.3’

SARS_CoV_2_vs_RTG13 100%[===================>]  59.52K  --.-KB/s    in 0.001s  

2022-07-13 17:21:37 (106 MB/s) - ‘SARS_CoV_2_vs_RTG13.fasta.3’ saved [60953/60953]



# Reading DNA Sequences

How do we actually read and interpret a genome? As we know, genomes consist of A,T,C, and G (for DNA genomes), and A, U, C, and G (for RNA genomes). However, note that when you download a genome from a public database, it is always given in terms of A, T, C, and G, regardless of if it is an RNA or DNA genome. This is just to help standardize data structures and analysis pipelines.

Collections of genomes are give in the form of files called `.fasta` files. We can read these files using a library called SeqIO from a collection of biology-related tools called BioPython.

**Let's read in some sequences!**

### Exercise: Reading DNA Sequences

We are going to read in two genomes:


*   A [SARS-CoV-2 genome](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512) from one of the first patients with COVID-19 in Wuhan.
*   A bat coronavirus, known as [RTG13](https://www.ncbi.nlm.nih.gov/nuccore/MN996532)

Use the form below to see what the genomes of the viruses look like as given in the `fasta` file.



In [ ]:
# This fasta file consists of 2 genomes.
# First is of the first SARS-CoV-2 sequenced from Wuhan (NC_k43lj)
# as well as the genome of the bat coronavirus that researchers believe 
# SARS-CoV-2 evolved from. 
sequences = [record for record in 
             SeqIO.parse(cov2_vs_rtg13,'fasta')]
sequence_bat = np.array(sequences[0])
sequence_cov2 = np.array(sequences[1])
virus = 'bat coronavirus (RTG13)' #@param ['bat coronavirus (RTG13)', 'SARS-CoV-2']
if virus=='SARS-CoV-2':
  print(sequences[1])
else:
  print(sequences[0])

ID: MN996532.1
Name: MN996532.1
Description: MN996532.1 Bat coronavirus RaTG13, complete genome
Number of features: 0
Seq('---------------CTTTCCAGGTAACAAACCAACGAACTCTCGATCTCTTGT...---')


### **Exercise: What do you think the symbol `-` in the sequences means?**

In [ ]:
Response = 'each dash represents a deletion that shows when compared to another deletion' #@param {type:"string"}
print("The -'s stand for deletions. ")
print("When we line up the two genomes against each other,")
print("sometimes there are gaps in the sequence, where DNA got deleted")
print("as the bat coronavirus slowly evolved to SARS-CoV-2.")
print("We will learn more about this in the BONUS colab if you are interested.")

The -'s stand for deletions. 
When we line up the two genomes against each other,
sometimes there are gaps in the sequence, where DNA got deleted
as the bat coronavirus slowly evolved to SARS-CoV-2.
We will learn more about this in the BONUS colab if you are interested.


# Visualizing DNA Sequences

There are many software packages avalable to visualize and browse genomes. Let's view these two genomes.
1. Navigate to NCBI's [Multiple Sequence Alignment Viewer tool](https://www.ncbi.nlm.nih.gov/projects/msaviewer/?appname=ncbi_msav&openuploaddialog). Click Under "Data Source", click "Text". 
2. Open up `SARS_CoV_2_vs_RTG13.fasta` in the Colab filesystem and copy/paste the contents into the textbox on the NCBI website. Click "upload" and then "close" and wait for your data to be uploaded.
3. Explore the data! Change the view on the "coloring" toolbar, and zoom in and out.

You should be able to get something that looks like the image below.

<a href="https://ibb.co/rx75s8j"><img src="https://i.ibb.co/ZBLcm4j/Screen-Shot-2020-05-18-at-4-44-17-PM.png" alt="Screen-Shot-2020-05-18-at-4-44-17-PM" border="0" /></a>
### **Exercise: Answer the following about the genome visualization in MSA Viewer.**

In [ ]:
#@title #### 1. What does the coloring mean?
Response = 'the coloring shows the different nucleotides within the genomes (A, C, G, T)' #@param {type:"string"}


In [ ]:
#@title #### 2. What does it mean when the two sequences have the same color vertical bar at the same location? Different colors?
Response = 'the same colors mean the same two nucleotides are at the same location, while different colors at the same location show different nucleotides at that location' #@param {type:"string"}

In [ ]:
#@title #### 3. How similar would you say the two viruses are?
Response = 'pretty similar' #@param {type:"string"}

# Parsing DNA Sequences
We can parse sequences by treating them similar to strings!  It is easiest to convert each sequence to a numpy array of individual letters. Use the form below to view different segments (from positions `start` to `end`) of the SARS-CoV-2 and bat coronavirus genome.

**Can you find regions where the two genomes are very different? Can you find regions where the two genomes are very similar?**

In [ ]:
sequence_bat = np.array(sequences[0])
sequence_cov2 = np.array(sequences[1])
start =  0#@param {type:"integer"}
stop =  1000#@param {type:"integer"}
print("SARS-CoV-2:")
print(''.join(sequence_cov2[start:stop]))
print("bat coronavirus:")
print(''.join(sequence_bat[start:stop]))
print(sum(sequence_cov2[start:stop]!=sequence_bat[start:stop]),
      "bases differ.")

SARS-CoV-2:
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACTGCACCTCATGGTCATGTTATGGTTGAGCTGGTAGCAGAACTCGAAGGCATTCAGTACGGTCGTAGTGGTGAGACACTTGGTGTCCTTGTCCCTCATGTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCTTCTTCGTAAGAACGGTAATAAAGGAGCTGGTGGCCATAGTTACGGCGCCGATCTAAAGTCATTTGACTTAGGCGACGAGCTTGGCACTGATCCTTATGAAGATTTTCAAGAAAACTGGAACACTAAACATAGCAGTGGTGTTACCCGTGAACTCATGCGTGAGCTTAACGGAGGGGCATACACTCGCTATGTCGATAACAACTTCTGTGGCCCTGATGGCTACCCTCTTGAGTGCATTAAAGACCTTCTAGCACGTGCTGGTAAAGCTTCATGCACTTTGTCCGAACAACTGGACTTTATTGACACTAAGAGGGGTGTATACTGCTGCCGTGAACATGAGCATGAAATTGCTTGGTAC

### **Exercise: Quantifying how similar the SARS-CoV-2 and the RTG13 bat coronavirus are.**
Fill in the code below to perfor the following.
1. Print the total number of bases in SARS-CoV-2 genome (Hint: You need to ignore `-`s!)
2. Print the number of bases (nucleotides) that differ between RTG13 and SARS-CoV-2.
3. Print the percent similarity between RTG13 and SARS-CoV-2.

In [ ]:
# Extract sequences.
n_bases_same = 0
sequence_bat = np.array(sequences[0]) 
sequence_cov2 = np.array(sequences[1])

# Compute length, number of different bases, 
# and number of the same bases.

#### Fill in code to compute the length of sars-cov-2 (ignore -'s if possible!) ########
length_cov2 = sum(sequence_cov2 != '-')
n_bases_different = sum(sequence_cov2 != sequence_bat)
#int(sum(sequence_bat) - int(sum(sequence_cov2)
n_bases_same == sum(sequence_cov2 == sequence_bat)

print("1. Length of SARS-CoV-2 genome: ", length_cov2)
print("2. # of bases that differ from RTG13: ", n_bases_different)
print("3. Percent similarity: %", 100*n_bases_same/length_cov2)

1. Length of SARS-CoV-2 genome:  29903
2. # of bases that differ from RTG13:  1189
3. Percent similarity: % 0.0


**Question: Now how similar do you think the two viruses are? Can you understand why scientists think SARS-CoV-2 originated from a bat?**

### **Exercise: Print the percent of A, T, C, and G in the SARS-CoV-2 genome.**

In [ ]:
num_A = sum(sequence_cov2=='A')
num_T = sum(sequence_cov2=='T')
num_C = sum(sequence_cov2=='C')
num_G = sum(sequence_cov2=='G')
num_hypen = sum(sequence_cov2=='-')
print("Percent A: %", (100*num_A/length_cov2))
print("Percent T: %", (100*num_T/length_cov2))
print("Percent C: %", (100*num_C/length_cov2))
print("Percent G: %", (100*num_G/length_cov2))
print(num_hypen)

Percent A: % 29.943483931378122
Percent T: % 32.083737417650404
Percent C: % 18.366050229074006
Percent G: % 19.60672842189747
0


**What do you notice about the percent of A/T/C/G?**  You can read more about G-C content [here](https://en.wikipedia.org/wiki/GC-content).

### **Exercise (BONUS): How many possible start codons does SARS-CoV-2 have? (ATG sequences)**

In [ ]:
n_start_codons = 0
for i in range(len(sequence_cov2[:-3])):
  codon = ''.join(sequence_cov2[i:(i+3)])
  if codon == (sequence_cov2 == 'ATG'):
    n_start_codons = n_start_codons + 1

print("Number of ATG start codons: ", n_start_codons)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  after removing the cwd from sys.path.


Number of ATG start codons:  0


**What do you notice about the number of start codons?** Even though SARS-CoV-2 has many possible start codons, it produces only around 10 canonical proteins!  You can read more about virus open reading frames [here](https://en.wikipedia.org/wiki/Reading_frame)


# More Exploring


### **Exercise: Viewing SARS-CoV-2 in UCSC genome browser.**
You can view an annotated version of the SARS-CoV-2 genome via the [UCSC genome browser](https://genome.ucsc.edu/cgi-bin/hgTracks?db=wuhCor1&lastVirtModeType=default&lastVirtModeExtraState=&virtModeType=default&virtMode=0&nonVirtPosition=&position=NC_045512v2%3A1%2D29903&hgsid=823607307_GTWpNMWmkRdDAUIlnTnE9U88LyB3). Play around for a few minutes. See if you can find the start codon that starts the S (spike) protein 1, ORF1a (non-structural proteins helping in replication), M (membrane) protein and the E (envelope) protein.

<a href="https://ibb.co/tqhjsc5"><img src="https://i.ibb.co/gS7qTwC/Screen-Shot-2020-05-18-at-5-31-48-PM.png" alt="Screen-Shot-2020-05-18-at-5-31-48-PM" border="0"></a><br />



What other interesting features do you notice about the SARS-CoV-2 genome? You can use UCSC Genome Browser, MSA Viewer, or python!

### **Exercise: List 3 interesting features you find**

In [ ]:
### Explore with python here #####


In [ ]:
_1_ = 'different amino acids have different colors' #@param {type:"string"}
_2_ = 'we can compare cross sequences by looking at mutations' #@param {type:"string"}
_3_ = 'we can see information by clicking on things' #@param {type:"string"}

# Wrapping up

***Awesome!*** Digging through genomes for clues about nature is a daunting task. But if done right, we can learn some fascinating things about how organisms function without having to lift a single test tube!


![](https://imgs.xkcd.com/comics/dna.png)